## Data Preprocessing 

In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler

# ===============================================================
# 1. LOAD ALL DATASETS
# ===============================================================

path = '/Users/satvikshankar/Downloads/archive (2)/'

main = pd.read_csv(path + "youtube_shorts_tiktok_trends_2025.csv_ML.csv")
country = pd.read_csv(path + "country_platform_summary_2025.csv")
monthly = pd.read_csv(path + "monthly_trends_2025.csv")
creators = pd.read_csv(path + "top_creators_impact_2025.csv")
hashtags = pd.read_csv(path + "top_hashtags_2025.csv")

print("Loaded datasets:")
print(main.shape, country.shape, monthly.shape, creators.shape, hashtags.shape)


# ===============================================================
# 2. BASIC CLEANING & TYPE FIXES
# ===============================================================

# Standardize column names (lowercase, snake_case)
def normalize_cols(df):
    df.columns = (
        df.columns.str.strip()
        .str.lower()
        .str.replace(" ", "_")
        .str.replace("-", "_")
    )
    return df

main = normalize_cols(main)
country = normalize_cols(country)
monthly = normalize_cols(monthly)
creators = normalize_cols(creators)
hashtags = normalize_cols(hashtags)

# Convert upload_date to datetime
if "upload_date" in main.columns:
    main["upload_date"] = pd.to_datetime(main["upload_date"], errors="coerce")

# Extract year, month for merging
main["month"] = main["upload_date"].dt.to_period("M").astype(str)


# ===============================================================
# 3. MERGING DATASETS
# ===============================================================

df = main.copy()

# Merge with country-level summary
if {"country", "platform"}.issubset(df.columns) and {"country", "platform"}.issubset(country.columns):
    df = df.merge(country, on=["country", "platform"], how="left")

# Merge with monthly trends
if "month" in df.columns and "month" in monthly.columns:
    df = df.merge(monthly, on="month", how="left")

# Merge with top creators (if creator_name exists)
creator_key = "creator_name"
if creator_key in df.columns and creator_key in creators.columns:
    df = df.merge(creators, on=creator_key, how="left")

# Merge hashtags (explode if multiple hashtags in a list)
if "hashtag" in hashtags.columns and "hashtags" in df.columns:
    df["hashtags"] = df["hashtags"].fillna("").astype(str)
    df_exp = df.assign(hashtag=df["hashtags"].str.split(",")).explode("hashtag")
    df = df_exp.merge(hashtags, on="hashtag", how="left").groupby("video_id").first().reset_index()

print("Merged dataset shape:", df.shape)


# ===============================================================
# 4. HANDLE MISSING VALUES
# ===============================================================

# Drop rows missing key identifiers
required_cols = ["video_id", "platform", "upload_date"]
df = df.dropna(subset=[c for c in required_cols if c in df.columns])

# Fill numeric columns
num_cols = df.select_dtypes(include=["float64", "int64"]).columns
df[num_cols] = df[num_cols].fillna(df[num_cols].median())

# Fill categorical columns
cat_cols = df.select_dtypes(include=["object"]).columns
df[cat_cols] = df[cat_cols].fillna("Unknown")


# ===============================================================
# 5. REMOVE DUPLICATES
# ===============================================================

if {"video_id", "platform"}.issubset(df.columns):
    df = df.drop_duplicates(subset=["video_id", "platform"])


# ===============================================================
# 6. OUTLIER HANDLING (IQR method)
# ===============================================================

for col in num_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    df[col] = np.where(df[col] < lower, lower, df[col])
    df[col] = np.where(df[col] > upper, upper, df[col])


# ===============================================================
# 7. NORMALIZE ENGAGEMENT METRICS
# ===============================================================

eng_cols = [c for c in df.columns if c in ["views", "likes", "shares", "comments"]]

for col in eng_cols:
    df[col + "_per_view"] = df[col] / (df["views"] + 1) if "views" in df.columns else df[col]


# ===============================================================
# 8. FEATURE ENCODING
# ===============================================================

# One-hot encode key categories
onehot_cols = ["platform", "language", "category", "sound_type", "country"]
onehot_cols = [c for c in onehot_cols if c in df.columns]

df = pd.get_dummies(df, columns=onehot_cols, drop_first=True)

# Label encode smaller categories
if "trend_label" in df.columns:
    le = LabelEncoder()
    df["trend_label_encoded"] = le.fit_transform(df["trend_label"])


# ===============================================================
# 9. STANDARD SCALING FOR NUMERIC FEATURES
# ===============================================================

scaler = StandardScaler()

scaled_cols = df.select_dtypes(include=["float64", "int64"]).columns
df[scaled_cols] = scaler.fit_transform(df[scaled_cols])

print("Final preprocessed dataset shape:", df.shape)

# Your final dataset:
final_df = df


Loaded datasets:
(50000, 32) (60, 14) (480, 8) (1000, 20) (82, 18)


KeyError: 'upload_date'

In [7]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler

# ===============================================================
# 1. LOAD ALL DATASETS
# ===============================================================

path = '/Users/satvikshankar/Downloads/archive (2)/'

main = pd.read_csv(path + "youtube_shorts_tiktok_trends_2025.csv_ML.csv")
country = pd.read_csv(path + "country_platform_summary_2025.csv")
monthly = pd.read_csv(path + "monthly_trends_2025.csv")
creators = pd.read_csv(path + "top_creators_impact_2025.csv")
hashtags = pd.read_csv(path + "top_hashtags_2025.csv")

print("Loaded datasets:")
print(main.shape, country.shape, monthly.shape, creators.shape, hashtags.shape)


# ===============================================================
# 2. BASIC CLEANING & TYPE FIXES
# ===============================================================

def normalize_cols(df):
    df.columns = (
        df.columns.str.strip()
        .str.lower()
        .str.replace(" ", "_")
        .str.replace("-", "_")
    )
    return df

main = normalize_cols(main)
country = normalize_cols(country)
monthly = normalize_cols(monthly)
creators = normalize_cols(creators)
hashtags = normalize_cols(hashtags)


# ===============================================================
# 3. MERGING DATASETS (NO TIMESTAMP MERGE)
# ===============================================================

df = main.copy()

# Merge with country summary if possible
if {"region", "platform"}.issubset(df.columns) and {"country", "platform"}.issubset(country.columns):
    df = df.merge(country, left_on=["region", "platform"], right_on=["country", "platform"], how="left")

# Merge monthly trends ONLY if "trend_label" matches monthly trend category
if "trend_label" in df.columns and "trend_label" in monthly.columns:
    df = df.merge(monthly, on="trend_label", how="left")

# Merge creators if matching key exists
if "creator_tier" in df.columns and "creator_tier" in creators.columns:
    df = df.merge(creators, on="creator_tier", how="left")

# Merge hashtags if you have hashtag interaction metrics
if "hashtag" in hashtags.columns:
    # This dataset has no raw hashtags column so skip
    pass

print("Merged dataset shape:", df.shape)


# ===============================================================
# 4. HANDLE MISSING VALUES
# ===============================================================

# Drop rows missing minimal keys
required_cols = ["platform", "category"]
df = df.dropna(subset=[c for c in required_cols if c in df.columns])

# Numeric fill
num_cols = df.select_dtypes(include=["float64", "int64"]).columns
df[num_cols] = df[num_cols].fillna(df[num_cols].median())

# Categorical fill
cat_cols = df.select_dtypes(include=["object"]).columns
df[cat_cols] = df[cat_cols].fillna("Unknown")


# ===============================================================
# 5. REMOVE DUPLICATES
# ===============================================================

df = df.drop_duplicates()


# ===============================================================
# 6. OUTLIER HANDLING (IQR method)
# ===============================================================

for col in num_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    df[col] = np.where(df[col] < lower, lower, df[col])
    df[col] = np.where(df[col] > upper, upper, df[col])


# ===============================================================
# 7. NORMALIZE ENGAGEMENT (already normalized, but adding relative forms)
# ===============================================================

eng_cols = ["like_rate", "comment_rate", "share_rate"]
eng_cols = [c for c in eng_cols if c in df.columns]

for col in eng_cols:
    df[col + "_zscore"] = (df[col] - df[col].mean()) / df[col].std()


# ===============================================================
# 8. FEATURE ENCODING
# ===============================================================

# One-hot encode key categories
onehot_cols = ["platform", "region", "language", "category", "traffic_source", "device_brand"]

onehot_cols = [c for c in onehot_cols if c in df.columns]

df = pd.get_dummies(df, columns=onehot_cols, drop_first=True)

# Label encode the trend label
if "trend_label" in df.columns:
    le = LabelEncoder()
    df["trend_label_encoded"] = le.fit_transform(df["trend_label"])
# ===== CLEANUP BEFORE STANDARD SCALER =====

# Replace inf
df = df.replace([np.inf, -np.inf], np.nan)

# Fill remaining NaNs in numeric columns
df[df.select_dtypes(include=[np.number]).columns] = \
    df.select_dtypes(include=[np.number]).fillna(df.median(numeric_only=True))

# Remove zero-variance columns
zero_var_cols = [col for col in df.select_dtypes(include=[np.number]).columns 
                 if df[col].nunique() <= 1]

print("Dropping zero-variance columns:", zero_var_cols)
df = df.drop(columns=zero_var_cols)

# Standard scaling
scaler = StandardScaler()
num_cols = df.select_dtypes(include=["float64", "int64"]).columns
df[num_cols] = scaler.fit_transform(df[num_cols])


# ===============================================================
# 9. STANDARD SCALING FOR NUMERIC FEATURES
# ===============================================================

scaler = StandardScaler()
scaled_cols = df.select_dtypes(include=["float64", "int64"]).columns

df[scaled_cols] = scaler.fit_transform(df[scaled_cols])

print("Final preprocessed dataset shape:", df.shape)

final_df = df

output_path = "/Users/satvikshankar/Downloads/youtube_tiktok_trends_2025_preprocessed.csv"

final_df.to_csv(output_path, index=False)

print("Saved preprocessed dataset to:", output_path)


Loaded datasets:
(50000, 32) (60, 14) (480, 8) (1000, 20) (82, 18)
Merged dataset shape: (1400094, 52)
Dropping zero-variance columns: ['weekend_hashtag_boost', 'total_videos', 'total_views', 'median_er', 'p95_views', 'avg_duration', 'avg_velocity_x', 'avg_comment_ratio', 'avg_share_rate', 'avg_save_rate', 'avg_engagement_per_1k', 'top_hashtag_views']
Final preprocessed dataset shape: (1400094, 76)
Saved preprocessed dataset to: /Users/satvikshankar/Downloads/youtube_tiktok_trends_2025_preprocessed.csv
